## Downloading the xmls from the dois

In [1]:
import pandas as pd
import requests
csv = pd.read_csv('scopus.csv')
df = pd.DataFrame(csv)

dois =df['DOI'].astype(str)

In [3]:

status_code = []
j =1
for i, doi in enumerate(dois):
  xml_url='https://api.elsevier.com/content/article/doi/' + doi + '?view=FULL'

  headers = {
            'X-ELS-APIKEY': 'c8cf76a7333ef42cbb26d1ce3aadc8bd',
            'Accept': 'text/xml'
          }

  r = requests.get(xml_url,stream = True,  headers=headers, timeout=30)

  status_code.append(r.status_code)
  if r.status_code == 200:
    try:
      writefile  = open("H:/My Drive/SMA/Clean/xmls/" + str(j) + ".xml", 'wb')



      for chunk in r.iter_content(2048):
        writefile.write((chunk))
    except:
      continue
  j = j+1

df['StatusCode'] = status_code

df.to_csv('scopus.csv',index=False)

In [ ]:
df.to_csv('scopus.csv',index=False)

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import csv
import re

for i,row in df.iterrows():
    status_code = row['StatusCode']
    if status_code == 200:
        # ENTER THE LOCTION OF DOWNLOADED XML FILE
        xml_file = f'G:\My Drive\IIT Kanpur\Summer_Project\DOI\sma_{i+1}.xml'
        tree = ET.parse(xml_file)
        root = tree.getroot()


        images = []
        captions = []


        for figure in root.findall('.//ce:figure', namespaces={'ce': 'http://www.elsevier.com/xml/common/dtd'}):
            link_element = figure.find('.//ce:link', namespaces={'ce': 'http://www.elsevier.com/xml/common/dtd'})
            locator = link_element.attrib.get('locator') if link_element is not None else None

            caption_element = figure.find('.//ce:caption', namespaces={'ce': 'http://www.elsevier.com/xml/common/dtd'})
            if caption_element is not None:
                simple_para_element = caption_element.find('.//ce:simple-para', namespaces={'ce': 'http://www.elsevier.com/xml/common/dtd'})
                if simple_para_element is not None:

                    raw_caption = ET.tostring(simple_para_element, encoding='unicode', method='text')
                    caption = re.sub('<.*?>', '', raw_caption)
                    caption = " ".join(caption.split())
                else:
                    caption = None
                captions.append(caption)
            else:
                captions.append(None)

        for attachment in root.findall('.//xocs:attachment', namespaces={'xocs': 'http://www.elsevier.com/xml/xocs/dtd'}):
            attachment_eid = attachment.find('.//xocs:attachment-eid', namespaces={'xocs': 'http://www.elsevier.com/xml/xocs/dtd'}).text
            ucs_locator = attachment.find('.//xocs:ucs-locator', namespaces={'xocs': 'http://www.elsevier.com/xml/xocs/dtd'}).text
            filename = attachment.find('.//xocs:filename', namespaces={'xocs': 'http://www.elsevier.com/xml/xocs/dtd'}).text
            images.append((attachment_eid, ucs_locator, filename))


        with open(f'output{i+1}.csv', 'w', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['UTD EID', 'UCS Locator', 'Filename', 'Caption'])
            for image, caption in zip(images, captions):
                csv_writer.writerow([image[0], image[1], image[2], caption])

        df_output = pd.read_csv(f'output{i+1}.csv')
        df_output['Article Link'] = df['Link'][i]
        df_output['link'] = df_output['UTD EID'].apply(lambda x: f'https://ars.els-cdn.com/content/image/{x}'.replace('.jpg', '_lrg.jpg'))
        df_output.to_csv(f'output{i+1}.csv', index=False)


## Extracting the caption along with the links of the images

In [ ]:
##another code
import xml.etree.ElementTree as ET
import pandas as pd
import csv
import re

# Assuming 'df' is already defined and read from 'scopus2.csv'
for i, row in df.iterrows():
    status_code = row['StatusCode']
    if status_code == 200:
        print(i+1)
        # reads the XML file with status code = 200
        # Change the Path for your file
        xml_file = f'G:\My Drive\IIT Kanpur\Summer_Project\DOI\sma_{i + 1}.xml'

        try:
            tree = ET.parse(xml_file)
            root = tree.getroot()

            images = []
            captions = []

            # Code to obtain image and captions from the XML file
            for figure in root.findall('.//ce:figure', namespaces={'ce': 'http://www.elsevier.com/xml/common/dtd'}):
                link_element = figure.find('.//ce:link', namespaces={'ce': 'http://www.elsevier.com/xml/common/dtd'})
                locator = link_element.attrib.get('locator') if link_element is not None else None

                caption_element = figure.find('.//ce:caption', namespaces={'ce': 'http://www.elsevier.com/xml/common/dtd'})
                if caption_element is not None:
                    simple_para_element = caption_element.find('.//ce:simple-para', namespaces={'ce': 'http://www.elsevier.com/xml/common/dtd'})
                    if simple_para_element is not None:
                        raw_caption = ET.tostring(simple_para_element, encoding='unicode', method='text')
                        caption = re.sub('<.*?>', '', raw_caption)
                        caption = " ".join(caption.split())
                    else:
                        caption = None
                    captions.append(caption)
                else:
                    captions.append(None)

            for attachment in root.findall('.//xocs:attachment', namespaces={'xocs': 'http://www.elsevier.com/xml/xocs/dtd'}):
                attachment_eid = attachment.find('.//xocs:attachment-eid', namespaces={'xocs': 'http://www.elsevier.com/xml/xocs/dtd'}).text
                ucs_locator = attachment.find('.//xocs:ucs-locator', namespaces={'xocs': 'http://www.elsevier.com/xml/xocs/dtd'}).text
                filename = attachment.find('.//xocs:filename', namespaces={'xocs': 'http://www.elsevier.com/xml/xocs/dtd'}).text
                images.append((attachment_eid, ucs_locator, filename))

        # Below code appends images and captions obtained in the output{i+1}.csv file
        # Here UTD EID refers to the image link suffix eg: "1-s2.0-S0020740324002984-gr7.jpg"
        # UCS Locator refers to the image link eg : "https://s3-eu-west-1.amazonaws.com/prod-ucs-content-store-eu-west/content/pii:S0020740324002984/gr7/\DOWNSAMPLED/image/jpeg/3490b0cff97734bba40bd0e7724986c9/gr7.jpg

            with open(f'output{i + 1}.csv', 'w', newline='', encoding='utf-8') as csvfile:
                csv_writer = csv.writer(csvfile)
                csv_writer.writerow(['UTD EID', 'UCS Locator', 'Filename', 'Caption'])
                for image, caption in zip(images, captions):
                    csv_writer.writerow([image[0], image[1], image[2], caption])


            df_output = pd.read_csv(f'output{i + 1}.csv')
        # Replacing '.jpg' with '_lrg.jpg' gives us a high resolution image
            df_output['link'] = df_output['UTD EID'].apply(lambda x: f'https://ars.els-cdn.com/content/image/{x}'.replace('.jpg', '_lrg.jpg'))
            df_output['Article Link'] = df['Link'][i]
            df_output.to_csv(f'output{i + 1}.csv', index=False)

            except ET.ParseError as e:
                print(f"ParseError: {e}. The file{xml_file} might be incomplete or malformed.")
            except Exception as e:
                print(f"An error occurred while processing the file{xml_file}: {e}")


## Binary Classification Part


In [ ]:
import os
import re
import pandas as pd

df = pd.read_csv('scopus2.csv')

# Keywords to be highlighted
keywords = [
    'strain', 'DSC', 'stress', 'heat flow', 'displacement', 'force',
    'martensite', 'fraction', 'load', 'X-Ray', 'temp', 'Differential',
    'scanning', 'calorimetry', 'curve'
]

# Function to highlight keywords in the caption
def highlight_keywords(caption, keywords):
    if not isinstance(caption, str):
        caption = ''

    found = False
    found_keywords = []
    for keyword in keywords:
        if re.search(r'\b' + re.escape(keyword) + r'\b', caption, re.IGNORECASE):
            found = True
            found_keywords.append(keyword)
            caption = re.sub(r'(?i)\b' + re.escape(keyword) + r'\b', r'**\g<0>**', caption)
    return found, caption, found_keywords

# Loop through the rows in the main DataFrame
for i, row in df.iterrows():
    status_code = row['StatusCode']

    # Proceed only if the status code is 200
    if status_code == 200:
        output_file = f'output{i+1}.csv'

        # Check if the file exists
        if os.path.exists(output_file):
            try:
                # Read the corresponding output CSV file
                dof = pd.read_csv(output_file)

                # Apply the keyword highlighting function
                results = dof['Caption'].apply(lambda x: highlight_keywords(x, keywords))
                dof['contains_keywords'] = results.apply(lambda x: 1 if x[0] else 0)
                # dof['highlighted_caption'] = results.apply(lambda x: x[1])
                dof['found_keywords'] = results.apply(lambda x: ', '.join(x[2]) if x[2] else '')

                # Save the updated DataFrame back to the CSV file
                dof.to_csv(output_file, index=False)

            except Exception as e:
                print(f"An error occurred while processing the file {output_file}: {e}")
        else:
            print(f"The file {output_file} does not exist.")


The file output199.csv does not exist.


#####  Since we have implemented the code for 198 files hence it stopped

## Downloading the Images which are marked as 1


In [ ]:
import os
import pandas as pd
import requests

# Read the main DataFrame
df = pd.read_csv('scopus2.csv')

# Loop through the rows in the main DataFrame
for i, row in df.iterrows():
    status_code = row['StatusCode']

    # Proceed only if the status code is 200
    if status_code == 200:
        output_file = f'output{i+1}.csv'

        # Check if the corresponding output CSV file exists
        if os.path.exists(output_file):
            dof = pd.read_csv(output_file)

            # Filter the DataFrame to get rows where 'contains_keywords' is 1
            contain = dof[dof['contains_keywords'] == 1]

            # Check if there are any rows to process
            if not contain.empty:
                # Define the directory to save the images
                loc = f'H:/My Drive/SMA/Clean/images/{i+1}'
                os.makedirs(loc, exist_ok=True)

                # Variable to track if any file was downloaded
                files_downloaded = False

                # Loop through the filtered rows
                for i1, r2 in contain.iterrows():
                    url = r2['link']

                    try:
                        response = requests.get(url, stream=True, timeout=30)

                        # Check if the request was successful
                        if response.status_code == 200:
                            # Extract the file name from the URL
                            file_name = url.split("/")[-1]

                            # Define the complete path for saving the file
                            save_path = os.path.join(loc, file_name)

                            # Write the file data to a file
                            with open(save_path, 'wb') as file:
                                for chunk in response.iter_content(1024):
                                    file.write(chunk)
                            print(f"File downloaded: {save_path}")
                            files_downloaded = True
                        else:
                            print(f"Failed to download file: {url} (Status code: {response.status_code})")
                    except Exception as e:
                        print(f"Error downloading file: {url} ({e})")

                # If no files were downloaded, remove the created directory
                if not files_downloaded:
                    os.rmdir(loc)
                    print(f"Removed empty directory: {loc}")
        else:
            print(f"Output file not found: {output_file}")


Error downloading file: https://ars.els-cdn.com/content/image/1-s2.0-S1359645424003422-ga1_lrg.jpg (HTTPSConnectionPool(host='ars.els-cdn.com', port=443): Read timed out. (read timeout=30))
File downloaded: H:/My Drive/SMA/Clean/images/1\1-s2.0-S1359645424003422-gr8_lrg.jpg
File downloaded: H:/My Drive/SMA/Clean/images/1\1-s2.0-S1359645424003422-gr3_lrg.jpg
File downloaded: H:/My Drive/SMA/Clean/images/1\1-s2.0-S1359645424003422-gr1_lrg.jpg
File downloaded: H:/My Drive/SMA/Clean/images/1\1-s2.0-S1359645424003422-gr11_lrg.jpg
File downloaded: H:/My Drive/SMA/Clean/images/1\1-s2.0-S1359645424003422-gr12_lrg.jpg
File downloaded: H:/My Drive/SMA/Clean/images/1\1-s2.0-S1359645424003422-gr13_lrg.jpg
File downloaded: H:/My Drive/SMA/Clean/images/1\1-s2.0-S1359645424003422-gr9_lrg.jpg
File downloaded: H:/My Drive/SMA/Clean/images/1\1-s2.0-S1359645424003422-gr6_lrg.jpg
File downloaded: H:/My Drive/SMA/Clean/images/1\1-s2.0-S1359645424003422-gr7_lrg.jpg
File downloaded: H:/My Drive/SMA/Clean/ima

# 2nd Level Classification


#### To implement this code you need to download Tesseract-OCR file in your local device

In [ ]:
import re
import pandas as pd
from PIL import Image, ImageEnhance
import pytesseract
import os


# Path to the Tesseract OCR executable
# Paste the path of the downloaded Tesseract-OCR file here
pytesseract.pytesseract.tesseract_cmd = r"C:/Program Files/Tesseract-OCR/tesseract.exe"

def highlight_keywords(caption, keywords):
    if not isinstance(caption, str):
        caption = ''

    found = False
    for keyword in keywords:
        if re.search(r'\b' + re.escape(keyword) + r'\b', caption, re.IGNORECASE):
            found = True
            caption = re.sub(r'(?i)\b' + re.escape(keyword) + r'\b', r'**\g<0>**', caption)
    return found, caption

def process_image(img_path, keywords):
    try:
        image = Image.open(img_path)
        image = image.convert('L')
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(2)
        text = pytesseract.image_to_string(image)
        found, highlighted_text = highlight_keywords(text, keywords)
        return 2 if found else 0, text
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")
        return 1, ""

def process_row(row, img_path, keywords):
    if row['contains_keywords'] == 1:
        contains_keywords2, text = process_image(img_path, keywords)
        row['contains_keywords2'] = contains_keywords2
        row['text'] = text
    return row

def main():
    df = pd.read_csv('scopus2.csv')

    for i, row in df.iterrows():
        if row['StatusCode'] == 200:
            try:
                dof = pd.read_csv(f'output{i+1}.csv')
                print(f"Processing output{i+1}.csv")

                img_dir = f"H:/My Drive/SMA/Clean/images/{i+1}/"
                dof = dof.apply(
                    lambda dof_row: process_row(dof_row, os.path.join(img_dir, dof_row['UTD EID'][:-4] + '_lrg.jpg'), keywords),
                    axis=1
                )

                dof.to_csv(f'output{i+1}.csv', index=False)
                print(f"Saved processed data to output{i+1}.csv")

            except FileNotFoundError as fnf_error:
                print(f"Error: {fnf_error}. Skipping processing of output{i+1}.csv")

            except Exception as e:
                print(f"Unexpected error processing output{i+1}.csv: {e}")

    df.to_csv('scopus2.csv', index=False)
    print("Main data frame updated and saved.")

if __name__ == "__main__":
    keywords = [
        'strain', 'DSC', 'stress', 'heat flow', 'displacement', 'force',
        'martensite', 'fraction', 'load', 'X-Ray', 'temp', 'Differential',
        'scanning', 'calorimetry', 'curve'
    ]
    main()


Processing output1.csv
Saved processed data to output1.csv
Processing output2.csv
Saved processed data to output2.csv
Processing output3.csv
Saved processed data to output3.csv
Processing output4.csv
Saved processed data to output4.csv
Processing output5.csv
Saved processed data to output5.csv
Processing output6.csv
Saved processed data to output6.csv
Processing output7.csv
Saved processed data to output7.csv
Processing output9.csv
Saved processed data to output9.csv
Processing output10.csv
Saved processed data to output10.csv
Processing output11.csv
Saved processed data to output11.csv
Processing output13.csv
Saved processed data to output13.csv
Processing output14.csv
Saved processed data to output14.csv
Processing output15.csv
Saved processed data to output15.csv
Processing output16.csv
Saved processed data to output16.csv
Processing output18.csv
Saved processed data to output18.csv
Processing output19.csv
Saved processed data to output19.csv
Processing output20.csv
Saved processed 

# Refined Images


In [ ]:
import pandas as pd
import os
import shutil
import winshell

def main():
    df = pd.read_csv('scopus2.csv')
    count = 0

    for i, row in df.iterrows():
        df.at[i, 'Processed'] = 0
        if row['StatusCode'] == 200:
            try:
                dof = pd.read_csv(f'output{i+1}.csv')
                dof['Article link'] = df['Link'][i]
                print(f"Processing output{i+1}.csv")

                img_dir = f"H:/My Drive/SMA/Clean/images/{i+1}/"
                refined_img_dir = f"H:/My Drive/SMA/Clean/images/Refined images/{i+1}/"

                # Check if there are rows in dof that meet the condition
                if len(dof) > 0 and any(dof['contains_keywords2'] == 2):
                    os.makedirs(refined_img_dir, exist_ok=True)

                    copied = False
                    for index, r1 in dof.iterrows():
                        try:
                            if r1['contains_keywords2'] == 2:
                                count += 1
                                src_img = os.path.join(img_dir, r1['UTD EID'][:-4] + '_lrg.jpg')
                                dst_img = os.path.join(refined_img_dir, r1['UTD EID'][:-4] + '_lrg.jpg')
                                shutil.copy(src_img, dst_img)
                                copied = True
                        except Exception as e:
                            print(f"Error processing row {index}: {e}")

                    # Check if any image was copied
                    if copied:
                        # Change the path of the folder according to your file location
                        firstloc = f"H:/My Drive/SMA/Clean/xmls/{i+1}.xml"
                        secondloc = f"H:/My Drive/SMA/Clean/xmls/output{i+1}.csv"
                        shutil.copy(firstloc, refined_img_dir)
                        shutil.copy(secondloc, refined_img_dir)

                        # Update scopus2.csv to mark this row as processed
                        df.at[i, 'Processed'] = 1

                        # Create a shortcut in the refined image directory
                        link = row['Link']
                        shortcut_path = os.path.join(refined_img_dir, f"Link_{i+1}.lnk")

                        # Create a shortcut
                        with winshell.shortcut(shortcut_path) as link_file:
                            link_file.path = link
                            link_file.description = f"Shortcut to {link}"
                            link_file.icon_location = (link, 0)  # You can customize the icon location

                dof.to_csv(f'output{i+1}.csv', index=False)

            except FileNotFoundError:
                print(f"FileNotFoundError: output{i+1}.csv not found.")
            except Exception as ex:
                print(f"Error processing output{i+1}.csv: {ex}")

    print(f"Total images copied: {count}")

    # Save the updated scopus2.csv with the 'Processed' column
    df.to_csv('scopus2.csv', index=False)

if __name__ == "__main__":
    main()


Processing output1.csv
Processing output2.csv
Processing output3.csv
Processing output4.csv
Processing output5.csv
Error processing output5.csv: 'contains_keywords2'
Processing output6.csv
Processing output7.csv
Processing output9.csv
Processing output10.csv
Error processing output10.csv: 'contains_keywords2'
Processing output11.csv
Processing output13.csv
Processing output14.csv
Processing output15.csv
Processing output16.csv
Processing output18.csv
Processing output19.csv
Processing output20.csv
Error processing output20.csv: 'contains_keywords2'
Processing output21.csv
Processing output22.csv
Processing output23.csv
Processing output25.csv
Error processing output25.csv: 'contains_keywords2'
Processing output26.csv
Processing output27.csv
Processing output28.csv
Processing output29.csv
Processing output30.csv
Processing output31.csv
Processing output32.csv
Processing output33.csv
Processing output35.csv
Processing output36.csv
Processing output37.csv
Processing output38.csv
Processin

# XLABEL AND YLABEL


In [ ]:
import pandas as pd
import cv2
import pytesseract
import os

def extract_axis_labels(image_path):
    # Read the image
    img = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area, descending
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    x_axis_label = ""
    y_axis_label = ""

    # Assume the largest contour is the graph area
    if contours:
        graph_contour = contours[0]
        x, y, w, h = cv2.boundingRect(graph_contour)

        # Extract region below the graph for x-axis label
        x_label_region = gray[y+h:, x:x+w]
        x_axis_label = pytesseract.image_to_string(x_label_region).strip()

        # Extract region to the left of the graph for y-axis label
        y_label_region = gray[y:y+h, :x]
        y_axis_label = pytesseract.image_to_string(y_label_region).strip()

    return x_axis_label, y_axis_label

def main():
    df = pd.read_csv('scopus2.csv')

    for i, row in df.iterrows():
        if row['Processed'] == 1:
            try:
                img_path = f"H:/My Drive/SMA/Clean/images/Refined images/{i+1}/"

                # Read output{i+1}.csv to get images marked as 2
                output_csv_path = f'H:/My Drive/SMA/Clean/images/Refined images/{i+1}/output{i+1}.csv'
                dof = pd.read_csv(output_csv_path)

                # Add new columns for X-axis Label and Y-axis Label if not already present
                if 'X-axis Label' not in dof.columns:
                    dof['X-axis Label'] = ""
                if 'Y-axis Label' not in dof.columns:
                    dof['Y-axis Label'] = ""

                for index, r1 in dof.iterrows():
                    if r1['contains_keywords2'] == 2:
                        image_file = r1['UTD EID'][:-4] + '_lrg.jpg'
                        image_path = os.path.join(img_path, image_file)

                        x_label, y_label = extract_axis_labels(image_path)

                        # Update or use x_label and y_label as needed
                        dof.at[index, 'X-axis Label'] = x_label
                        dof.at[index, 'Y-axis Label'] = y_label

                # Save the updated output{i+1}.csv
                dof.to_csv(output_csv_path, index=False)

            except Exception as e:
                print(f"Error processing row {i+1}: {e}")

if __name__ == "__main__":
    main()


Error processing row 2: tile cannot extend outside image
Error processing row 18: tile cannot extend outside image
Error processing row 30: tile cannot extend outside image
Error processing row 49: tile cannot extend outside image
Error processing row 74: tile cannot extend outside image
Error processing row 91: tile cannot extend outside image
Error processing row 118: tile cannot extend outside image
Error processing row 129: tile cannot extend outside image
Error processing row 193: tile cannot extend outside image


#### for some images X-Y Labels were not obtained

#### We have implemented this code for a small data set of 200 DOIs we can implement it similarly for 1000+ DOIs, it took too much time for computation around 2.5-3 hours hence we thought to share the code and output for small data set